## Auto mode (single uav)

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [ ]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_home = ENUPose(5,5,0,45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc_config.show()

## No Visualizer

In [5]:
novis_config = ConfigVis[int]()
novis_config.add_vehicle(1)

# Save Mission

In [6]:
qgc_config.save_missions()

In [7]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [8]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz],
	verbose=2,
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1]})

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 1395718)
🚀 UAV logic for vehicle 1 launched (PID 1395720)
🚀 Proxy for vehicle 1 launched (PID 1395722)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at [-35.363328, 149.1652241, 0.0, 90.0]
SIM_VEHICLE: Using defaults from (../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=1
Starting SITL Gazebo
Home: -35.363328 149.165224 alt=0.000000m hdg=90.000000
Bind 127.0.0.1:9003 for SITL in
Setting Gazebo interface to 127.0.0.1:9002 
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9005
Waiting for connection ....


bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


System id: 1
Connection on serial port 5760
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 1

🚀 Starting Proxy 1


Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures


Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/def

## Oracle checking

In [9]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
Vehicle 1: 🧹 Cleared previous mission
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 1: ✅ 7 waypoints read.
Vehicle 1: 🧭 Mission[0] → cmd: WAYPOINT, x: -35.3632831, y: 149.165169, z: 0.0, current: 0
Vehicle 1: 🧭 Mission[1] → cmd: TAKEOFF, x: -35.3632831, y: 149.165169, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[2] → cmd: WAYPOINT, x: -35.3633466, y: 149.1650911, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[3] → cmd: WAYPOINT, x: -35.3632831, y: 149.1650132, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[4] → cmd: WAYPOINT, x: -35.3632196, y: 149.1650911, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[5] → cmd: WAYPOINT, x: -35.3632831, y: 149.165169, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[6] → cmd: LAND, x: -35.3632831, y: 149.165169, z: 0.0, current: 0
Vehicle 1: ✅ Sent mission item 0
Vehicle 1: ✅ Sent mission item 1
Vehicle 1: ✅ Sent mission it

📤 GCS ← UAV 1: Sending DONE (attempt 2)
📤 GCS ← UAV 1: Sending DONE (attempt 3)
📤 GCS ← UAV 1: Sending DONE (attempt 4)
📤 GCS ← UAV 1: Sending DONE (attempt 5)
📤 GCS ← UAV 1: Sending DONE (attempt 6)
📤 GCS ← UAV 1: Sending DONE (attempt 7)
📤 GCS ← UAV 1: Sending DONE (attempt 8)
📤 GCS ← UAV 1: Sending DONE (attempt 9)
✅ Vehicle 1 completed its mission
✅ All UAVs assigned to GCS blue 🟦 have completed their mission.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 1 logic stopped.
Connection reset or closed by peer on TCP socket
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
❎ Proxy 1 stopped.
Closed connection on SERIAL0
